Swiftのバージョン

In [0]:
class Util{
  static var swiftVersion: String{
    #if swift(>=5.0)
      return "Swift 5.0"

    #elseif swift(>=4.2)
      return "Swift 4.2"

    #elseif swift(>=4.1)
     return "Swift 4.1"

    #elseif swift(>=4.0)
      return "Swift 4.0"

    #elseif swift(>=3.2)
      return "Swift 3.2"

    #elseif swift(>=3.0)
      return "Swift 3.0"

    #endif

    return "unknown"
  }  
}

print("Swift version: \(Util.swiftVersion)")

Swift version: Swift 5.0


Pythonのバージョン

In [0]:
import TensorFlow
import Python

print("Python version: \(Python.version)")

let np = Python.import("numpy")

let a = np.array([[1, 2], [3, 4]])
let b = np.array([[5], [6]])
let c = np.matmul(a, b)

print(c)

Python version: 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
[[17]
 [39]]


XOR

In [1]:
// XOR

import Foundation
import TensorFlow

// XORのモデル
struct XOR: Layer {
  
  var layer1: Dense<Float>
  var layer2: Dense<Float>
  
//   // ここで宣言と代入もできる
//   var layer1 = Dense<Float>(inputSize: 2,  outputSize: hiddenSize, activation: sigmoid)
  
  init(hiddenSize:Int = 2){
    self.layer1 = Dense(inputSize: 2,  outputSize: hiddenSize,  activation: sigmoid)
    self.layer2 = Dense(inputSize: hiddenSize, outputSize: 1,  activation: sigmoid)
  }
  
  @differentiable
  func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
    return input.sequenced(in: context, through: layer1, layer2)
    
//    // 層を逐次繋ぐ場合
//    let h1 = l1.applied(to: input, in: context)
//    let h2 = l2.applied(to: h1, in: context)
//    return h2
    }
}


let x: Tensor<Float> = [[0, 0], [0, 1], [1, 0], [1, 1]]
let y: Tensor<Float> = [[0], [1], [1], [0]]

var model = XOR()
let optimizer = SGD<XOR, Float>(learningRate: 0.1)
let context = Context(learningPhase: .training)

var date = Date()
for epoch in 1...5000 {
  
  let dmodel = model.gradient { m -> Tensor<Float> in
    let t = m.applied(to: x, in: context)
    let loss = sigmoidCrossEntropy(logits: t, labels: y)
    return loss
 }
  
  optimizer.update(&model.allDifferentiableVariables, along: dmodel)
 
}

let diff = Date().timeIntervalSince(date)
print("elapsed_time: \(diff) sec")

let inference = round(model.inferring(from: x))
// print(inference)

for i: Int32 in 0 ..< 4 {
    print("x: \(x[i]), y: \(y[i]), inference: \(inference[i]), result:\(y[i] == inference[i] )")
}

//print(model)


elapsed_time: 12.815145015716553 sec
x: [0.0, 0.0], y: [0.0], inference: [0.0], result:true
x: [0.0, 1.0], y: [1.0], inference: [1.0], result:true
x: [1.0, 0.0], y: [1.0], inference: [1.0], result:true
x: [1.0, 1.0], y: [0.0], inference: [0.0], result:true
